In [1]:
%reload_ext autoreload
%autoreload 2

from _header_model import *

# device = "cpu"

print("MPS Availible:\t", torch.backends.mps.is_available())
print(f"Using device:\t {device}")

Matplotlib created a temporary cache directory at /tmp/matplotlib-qcu23ibd because the default path (/home/lfi/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


ModuleNotFoundError: No module named 'src'

---
### DataLoader

<!-- Create HF Dataset

```py
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'cath_id','temperature', 'replica'],
        num_rows: n
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'cath_id','temperature', 'replica'],
        num_rows: n
    })
})
```

```rust
input_ids: Amino Acid Sequence
attention_mask: Padding Mask
cath_id: cath identifier i.e. 1a0rP01
replica: replica numbner in {0, 1, 2, 3, 4}
temperature: temperature of trajectory in {320, 348, 379, 413, 450}
sequence: original sequence
pssm: PSSM as numpy array dim(20, L)
``` -->

In [2]:
t5_tokenizer = T5Tokenizer.from_pretrained(
    pretrained_model_name_or_path=BASE_MODEL,
    do_lower_case=False,
    use_fast=True,
    legacy=False,
)

dict_pssm = np.load(os.path.join('../', FILE_PATHS["pssm"], "dict_pssm.npy"), allow_pickle=True).item()


def pssm_to_hf_dataset(dict_pssm: dict, tokenizer: T5Tokenizer) -> Dataset:
    # todo add train/test split
    ds_dict = {"cath_id": [], "temperature": [], "replica": [], "sequence": [], "sequence_processed": [], "pssm": []}

    for key, value in dict_pssm.items():
        parts = key.split("|")
        ds_dict["cath_id"].append(parts[0])
        ds_dict["temperature"].append(int(parts[1]))
        ds_dict["replica"].append(parts[2])
        ds_dict["sequence"].append(parts[3])
        ds_dict["sequence_processed"].append(" ".join(parts[3]))
        ds_dict["pssm"].append(value.tolist())

    tokenized_sequences = tokenizer(
        text=ds_dict["sequence_processed"],
        padding=False,
        truncation=False,
        max_length=512,
    )
    ds = Dataset.from_dict(tokenized_sequences)
    for key, value in ds_dict.items():
        ds = ds.add_column(key, value)
    
    # ds = ds.map(lambda examples: {'pssm': [torch.tensor(pssm) for pssm in examples['pssm']]}, batched=True)

    return DatasetDict({"train": ds, "test": ds})


ds = pssm_to_hf_dataset(dict_pssm=dict_pssm, tokenizer=t5_tokenizer)
ds = ds.remove_columns(["cath_id", "replica", "sequence", 'sequence_processed', "temperature"])
ds = ds.rename_column("pssm", "labels")
# ds = ds.remove_columns("labels")

ds["train"] = ds["train"].select([0, 49])
ds["test"] = ds["test"].select([0, 49])

print(ds)

# i = 0
# print(len(ds["train"]["attention_mask"][i]), ":", *ds["train"]["input_ids"][i])
# print(len(ds["train"]["attention_mask"][i]), ':', *ds["train"]["attention_mask"][i])
# display(pd.DataFrame(ds["train"]["labels"][i]))
# print(type(torch.tensor(ds["train"]["labels"][i])))

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
})


---
### Model Loading and LoRA

In [3]:
t5_base_model, loading_info = T5EncoderModelForPssmGeneration.from_pretrained(
    pretrained_model_name_or_path=BASE_MODEL,
    output_loading_info=True,
    # device_map=device,
    # load_in_8bit=False,
    # custom_dropout_rate=0.1,
)

modules_to_save = ["classifier"]

lora_config = LoraConfig(
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q", "k", "v", "o"],
    bias="none",
    modules_to_save=loading_info['missing_keys'], # check if saving 'pssm_head' also works
)

t5_lora_model = peft.get_peft_model(t5_base_model, lora_config)
t5_lora_model.print_trainable_parameters()
print(loading_info)

Some weights of T5EncoderModelForPssmGeneration were not initialized from the model checkpoint at Rostlab/prot_t5_xl_uniref50 and are newly initialized: ['pssm_head.bn1.bias', 'pssm_head.bn1.num_batches_tracked', 'pssm_head.bn1.running_mean', 'pssm_head.bn1.running_var', 'pssm_head.bn1.weight', 'pssm_head.bn2.bias', 'pssm_head.bn2.num_batches_tracked', 'pssm_head.bn2.running_mean', 'pssm_head.bn2.running_var', 'pssm_head.bn2.weight', 'pssm_head.bn3.bias', 'pssm_head.bn3.num_batches_tracked', 'pssm_head.bn3.running_mean', 'pssm_head.bn3.running_var', 'pssm_head.bn3.weight', 'pssm_head.conv1.bias', 'pssm_head.conv1.weight', 'pssm_head.conv2.bias', 'pssm_head.conv2.weight', 'pssm_head.conv3.bias', 'pssm_head.conv3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 3,932,160 || all params: 1,214,057,788 || trainable%: 0.3239
{'missing_keys': ['pssm_head.bn1.bias', 'pssm_head.bn1.num_batches_tracked', 'pssm_head.bn1.running_mean', 'pssm_head.bn1.running_var', 'pssm_head.bn1.weight', 'pssm_head.bn2.bias', 'pssm_head.bn2.num_batches_tracked', 'pssm_head.bn2.running_mean', 'pssm_head.bn2.running_var', 'pssm_head.bn2.weight', 'pssm_head.bn3.bias', 'pssm_head.bn3.num_batches_tracked', 'pssm_head.bn3.running_mean', 'pssm_head.bn3.running_var', 'pssm_head.bn3.weight', 'pssm_head.conv1.bias', 'pssm_head.conv1.weight', 'pssm_head.conv2.bias', 'pssm_head.conv2.weight', 'pssm_head.conv3.bias', 'pssm_head.conv3.weight'], 'unexpected_keys': ['lm_head.weight'], 'mismatched_keys': [], 'error_msgs': []}


---
### Model Training

In [4]:
data_collator = DataCollatorForT5Pssm(
    tokenizer=t5_tokenizer,
    padding=True,
    max_length=512,
)

training_args = TrainingArguments(
    output_dir=FILE_PATHS["models"],
    learning_rate=TRAINING_CONFIG["learning_rate"],
    per_device_train_batch_size=TRAINING_CONFIG["batch_size"],
    per_device_eval_batch_size=TRAINING_CONFIG["batch_size"] * 2,
    num_train_epochs=TRAINING_CONFIG["num_epochs"],
    logging_steps=TRAINING_CONFIG["logging_steps"],
    evaluation_strategy="steps", # use eval_strategy
    eval_steps=TRAINING_CONFIG["eval_steps"],
    save_strategy="steps",
    save_steps=TRAINING_CONFIG["save_steps"],
    remove_unused_columns=True,
    # label_names=["labels"],
    seed=SEED,
)

trainer = Trainer(
    model=t5_lora_model,
    args=training_args,
    train_dataset=ds["train"],
    # eval_dataset=dataset_signalp['valid'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/Users/finnlueth/Developer/gits/bachelor-thesis/mdcath-to-3di/.venv/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
# print(*ds['train']['input_ids'], sep="\n")

# for name, param in t5_base_model.named_parameters():
#     print(name)

# t5_base_model.encoder.block[0].layer[0].SelfAttention.q.weight

In [ ]:
gc.collect()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

if torch.cuda.is_available():
    torch.cuda.empty_cache()
if torch.backends.mps.is_available():
    torch.mps.empty_cache()

trainer.train()

  0%|          | 0/10 [00:00<?, ?it/s]

no_labels_features [{'input_ids': [7, 5, 15, 9, 15, 23, 5, 18, 3, 8, 7, 5, 6, 12, 19, 17, 10, 7, 5, 3, 7, 11, 14, 7, 5, 3, 18, 12, 11, 13, 3, 5, 9, 11, 5, 5, 3, 12, 5, 8, 4, 5, 17, 16, 3, 10, 11, 18, 6, 9, 19, 17, 4, 9, 23, 14, 16, 11, 4, 10, 17, 5, 3, 11, 11, 8, 15, 14, 6, 19, 6, 3, 10, 5, 16, 11, 7, 18, 17, 10, 21, 11, 3, 7, 11, 7, 10, 4, 17, 6, 8, 16, 3, 15, 6, 9, 4, 5, 17, 4, 13, 11, 15, 3, 5, 13, 15, 14, 5, 7, 11, 4, 21, 3, 5, 14, 8, 15, 10, 8, 10, 17, 15, 10, 12, 23, 21, 12, 10, 7, 10, 6, 6, 15, 4, 3, 5, 11, 5, 5, 5, 12, 18, 10, 6, 14, 21, 17, 10, 5, 4, 8, 7, 17, 15, 7, 4, 18, 5, 8, 17, 15, 5, 10, 12, 10, 10, 7, 7, 17, 7, 6, 16, 17, 18, 12, 4, 11, 19, 17, 23, 15, 3, 5, 13, 4, 16, 19, 19, 6, 7, 5, 4, 8, 3, 14, 10, 17, 10, 9, 8, 14, 10, 7, 17, 5, 17, 4, 3, 14, 5, 10, 3, 3, 17, 11, 5, 6, 23, 3, 4, 4, 5, 4, 23, 17, 10, 7, 15, 18, 5, 4, 8, 10, 5, 7, 7, 14, 11, 3, 4, 4, 18, 5, 23, 5, 4, 5, 3, 9, 6, 14, 5, 12, 5, 7, 10, 5, 3, 4, 8, 13, 5, 3, 10, 11, 21, 8, 12, 3, 7, 18, 5, 11, 11, 13, 4

/Users/finnlueth/Developer/gits/bachelor-thesis/mdcath-to-3di/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2906: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412
0,-inf,1.632268,1.847997,0.201634,0.887525,2.711495,2.336283,2.419539,1.807355,0.378512,2.655352,1.137504,2.620586,1.827819,1.867896,1.632268,1.963474,1.459432,0.321928,0.807355,0.584962,0.201634,0.632268,1.321928,0.807355,0.765535,0.321928,0.378512,1.655352,0.201634,0.137504,-0.234465,-0.415038,0.000000,-1.321928,0.263034,-1.152003,-0.152003,1.292782,0.000000,0.137504,-1.000000,-1.152003,-0.514573,-0.234465,-0.736966,-0.234465,0.000000,0.584962,-0.321928,0.847997,0.070389,1.432959,0.137504,1.292782,0.485427,0.925999,1.608809,0.263034,0.321928,-1.514573,0.584962,0.722466,-0.234465,-0.074001,0.201634,1.232661,0.378512,1.201634,-0.415038,0.847997,-0.074001,0.432959,-0.514573,0.137504,0.584962,0.070389,0.321928,-0.074001,0.807355,1.070389,1.000000,0.963474,0.963474,0.584962,1.104337,-0.152003,0.378512,-0.074001,0.536053,1.321928,2.378512,1.137504,1.035624,-0.736966,0.536053,-0.152003,1.405992,1.000000,2.053111,1.536053,0.765535,0.678072,0.925999,2.247927,1.378512,0.765535,0.678072,0.765535,1.000000,1.169925,0.536053,1.263034,1.432959,1.070389,1.608809,0.887525,0.070389,0.432959,1.263034,0.432959,1.232661,-0.074001,0.847997,1.608809,0.847997,1.137504,-0.321928,0.485427,0.807355,-0.321928,0.536053,0.678072,0.201634,0.765535,1.405992,1.405992,1.201634,1.263034,0.485427,1.137504,1.137504,1.378512,0.887525,-0.152003,0.847997,0.263034,0.137504,-0.234465,-0.074001,0.378512,0.536053,0.807355,-0.074001,0.584962,0.807355,0.137504,0.765535,0.847997,0.765535,0.887525,1.678072,0.963474,-0.074001,0.263034,-0.862496,-1.514573,-0.736966,-1.321928,-1.514573,-0.074001,-0.736966,-0.514573,0.070389,-0.621488,-0.321928,-1.000000,-0.621488,0.678072,-0.415038,0.678072,0.765535,-0.321928,0.632268,1.070389,-0.862496,-0.234465,0.536053,-0.621488,0.378512,0.000000,0.485427,-0.152003,0.963474,-0.621488,-0.415038,-0.074001,-0.862496,-0.152003,0.137504,-0.621488,0.378512,-0.621488,0.378512,0.321928,0.536053,0.137504,0.963474,0.536053,1.070389,0.722466,0.378512,0.807355,0.432959,0.378512,1.485427,1.104337,0.765535,0.963474,0.632268,0.070389,1.000000,0.070389,-0.152003,0.584962,0.722466,0.000000,1.292782,0.485427,-0.321928,0.765535,-0.321928,0.263034,0.485427,0.201634,0.432959,0.000000,-0.234465,0.722466,0.584962,0.432959,0.378512,1.104337,0.678072,1.292782,0.807355,0.137504,-0.152003,-0.074001,-0.514573,-0.234465,0.963474,0.201634,0.678072,0.000000,-0.074001,-0.321928,-1.514573,-0.321928,-1.514573,-0.074001,-0.736966,-1.736966,-1.152003,-0.514573,

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412
0,-inf,0.632268,1.137504,0.070389,-1.736966,-2.736966,-2.736966,-0.736966,-1.514573,-0.415038,0.137504,-1.736966,-4.321928,-2.321928,-4.321928,-4.321928,-4.321928,-inf,-inf,-inf,-4.321928,-inf,-inf,-inf,-3.321928,-inf,-4.321928,-1.152003,-3.321928,-inf,-inf,-inf,-2.321928,1.378512,-inf,-2.321928,-0.736966,-inf,-inf,-3.321928,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-3.321928,-4.321928,-inf,-inf,-inf,-inf,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-10

KeyboardInterrupt: 

In [ ]:
# parent_class_name = T5EncoderModelForPssmGeneration.__bases__[0].__name__
# parent_class_name